In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import math
import time
import csv


In [7]:
df_aed = pd.read_parquet('aed_locations.parquet.gzip')

In [8]:
len(df_aed)

15227

In [9]:
#create address as one string 
number = 1
addresses = []
for number in range(len(df_aed)):
    address = ''
    if not pd.isna(df_aed['address'][number]):
        address = str(df_aed['address'][number])
        if not pd.isna(df_aed['number'][number]):
            address = address+" "+str(int(df_aed['number'][number]))
        if not pd.isna(df_aed['postal_code'][number]):
            address = address +" ,"+str(int(df_aed['postal_code'][number]))
        if not pd.isna(df_aed['municipality'][number]):
            address = address +" ,"+str(df_aed['municipality'][number])
        address = address + ' , Belgium'
        addresses.append(address)

In [28]:
def getting_coordinate(starting_value):
    counter = starting_value
    coordinates = []
    for i in range(120):
        some_locations = gpd.tools.geocode(addresses[counter:counter+10])
        some_coordinates = some_locations.get_coordinates()
        for index,row in some_coordinates.iterrows():
            coordinates.append(row)
        counter += 10
        time.sleep(1)
    return coordinates


In [33]:
len(coordinates)
#we have 1215 coordinates right now

1215

In [ ]:
#aed_coordinates1 = getting_coordinate(1)
aed_coordinates2 = getting_coordinate(120)
aed_coordinates3 = getting_coordinate(240)
aed_coordinates4 = getting_coordinate(360)
aed_coordinates5 = getting_coordinate(480)
aed_coordinates6 = getting_coordinate(600)
aed_coordinates7 = getting_coordinate(720)
aed_coordinates8 = getting_coordinate(840)
aed_coordinates9 = getting_coordinate(960)
aed_coordinates10 = getting_coordinate(1080)
aed_coordinates11 = getting_coordinate(1200)
aed_coordinates12 = getting_coordinate(1320)

In [ ]:
#change it to aed_coordinates2.csv etc.
file_path = 'aed_coordinates1.csv'

In [22]:
# Write the list to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(coordinates)

print("CSV file saved successfully.")

CSV file saved successfully.


In [34]:
column_names = ['lon', 'lat']

# Read the CSV file into a pandas DataFrame
aed_coordinates = pd.read_csv(file_path, names = column_names)


In [35]:
aed_coordinates.head()

,lon,lat
0,5.737995,50.612439
1,5.570912,50.646499
2,5.786602,50.285590
3,5.570912,50.646499
4,5.573780,50.637997


In [36]:
px.set_mapbox_access_token('pk.eyJ1Ijoia2V6aWFoZHV0dCIsImEiOiJjbHZnaXo3Y3cwcW16Mmpudnh6anRzZXp5In0.JawqtWmD9uBcGjDAt8C5zg')
center_lat = 50.5  # Latitude of the center of the country
center_lon = 4.3517  # Longitude of the center of the country
zoom_level = 7       # Zoom level (1-20)
# Create the base map
map_base = px.scatter_mapbox(lat=[center_lat], lon=[center_lon], zoom=zoom_level)

# Update the layout of the base map
map_base.update_layout(mapbox_style="light")

# Add the scatter plot layer with the coordinate locations
aed_location_layer = px.scatter_mapbox(aed_coordinates, lat='lat', lon='lon')

# Add the scatter plot layer to the base map
map_with_aed_points = map_base.add_trace(aed_location_layer.data[0])

# Show the map
map_with_aed_points.show()